# 4. Ensembles de Arboles de Decision

## 4.06 GBDT LightGBM

La técnica de Gradient Boosting fue creada por Jerome H. Friedman en 1999 - 2001
<br>Se implementaron librerías ineficientes
<br>En 2016 se crea XGBoost, en 2017 LightGBM




<br>Qué tipo de perturbaciones se realiza LightGBM

*   Se perturba el dataset, seleccionando para cada arbol un subconjunto de las columnas.
*   El algortimo de arbol de decisión no presenta perturbaciones

#### 4.06.1  Seteo del ambiente en Google Colab

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dm"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dm" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/itba2025-8d0a/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


### 4.07  LightGBM, una corrida

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

Aqui debe cargar SU semilla primigenia

In [ ]:
PARAM <- list()
PARAM$experimento <- 4070
PARAM$semilla_primigenia <- 163393

# CAMBIO CON VALORES OBTENIDOS POR OPTIMIZACION BAYESIANA
PARAM$lgb$num_iterations <- 2000  # cantidad de arbolitos
PARAM$lgb$learning_rate <- 0.1
PARAM$lgb$feature_fraction <- 0.49
PARAM$lgb$min_data_in_leaf <- 7474
PARAM$lgb$num_leaves <- 12
PARAM$lgb$max_bin <- 200


In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("KA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv", stringsAsFactors= TRUE)

In [ ]:
# paso la clase a binaria

dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2"), 1L, 0L)]

In [ ]:
# los campos que se van a utilizar

campos_buenos <- setdiff(colnames(dataset), c("clase_ternaria", "clase01"))

In [ ]:
# establezco donde entreno

dataset[, train := 0L]
dataset[foto_mes %in% c(202107), train := 1L]

In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[train == 1L, campos_buenos, with= FALSE]),
  label= dataset[train == 1L, clase01]
)

In [ ]:
# genero el modelo
# estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG") # Establezco la semilla aleatoria

modelo <- lgb.train(
  data= dtrain,
  param= list(
    objective= "binary",
    max_bin= PARAM$lgb$max_bin,
    learning_rate= PARAM$lgb$learning_rate,
    num_iterations= PARAM$lgb$num_iterations,
    num_leaves= PARAM$lgb$num_leaves,
    min_data_in_leaf= PARAM$lgb$min_data_in_leaf,
    feature_fraction= PARAM$lgb$feature_fraction,
    seed= PARAM$semilla_primigenia
  )
)


In [ ]:
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelo))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)


In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(modelo, "modelo.txt" )

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes == 202109]

# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)


In [ ]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

In [ ]:
# subidas a Kaggle
# ordeno por probabilidad descendente

setorder(tb_prediccion, -prob)

In [ ]:
# genero la prediccion y subo a Kaggle

tb_prediccion[, Predicted := 0L]
tb_prediccion[prob>(1/40), Predicted := 1L]

archivo_kaggle <- paste0("KA", PARAM$experimento, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

# subida a Kaggle
comando <- "kaggle competitions submit"
competencia <- "-c data-mining-analista-sr-2025-a"
arch <- paste( "-f", archivo_kaggle)

mensaje <- paste0("-m 'num_iterations=", PARAM$lgb$num_iterations,
  "  learning_rate=", PARAM$lgb$learning_rate,
  "  feature_fraction=", PARAM$lgb$feature_fraction,
  "  min_data_in_leaf=", PARAM$lgb$min_data_in_leaf,
  "  num_leaves=",PARAM$lgb$num_leaves,
  "  max_bin=", PARAM$lgb$max_bin,
"'" )

linea <- paste( comando, competencia, arch, mensaje)
salida <- system(linea, intern=TRUE)
cat(salida)